In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sb
import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import nltk

from nltk.corpus import stopwords

from sklearn.svm import SVC


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import PorterStemmer
import gensim
from gensim.models import Word2Vec 

In [2]:
df = pd.read_csv("./train.csv")
test=pd.read_csv('./test.csv')

In [3]:
col_namesssss = df.columns

col_namesssss

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [4]:
print(df.info())
print(df.head())
print(test.info())
print(test.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
None
   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  
<class 'pandas.core.frame.DataFrame'

In [5]:
classes = df["target"]
print(classes.value_counts())

target
0    4342
1    3271
Name: count, dtype: int64


In [6]:
df["text"].isnull().sum()
test["text"].isnull().sum()

0

In [7]:
df.fillna("none", inplace=True)
test.fillna("none", inplace=True)

In [8]:
# Replace URLs with 'webaddress'
df['text'] = df['text'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')
test['text'] = test['text'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')                                

In [9]:
# threshold = 0.005  # You can adjust this threshold

# # Calculate the frequency of each value
# value_counts = df['location'].value_counts()

# # Identify values to group into "Other"
# values_to_group = value_counts[value_counts / len(df) < threshold].index

# # Replace values in the DataFrame with "Other"
# df['location'] = df['location'].apply(lambda x: x if x not in values_to_group else 'Other')

# # Perform one-hot encoding
# one_hot = pd.get_dummies(df['location'], prefix='location')

# # Concatenate the one-hot encoded DataFrame with the original DataFrame
# df = pd.concat([df, one_hot], axis=1)

# # Drop the original 'countries' column
df.drop(['location','keyword'], axis=1, inplace=True)
test.drop(['location','keyword'], axis=1, inplace=True)

In [10]:
print(df.shape)
df.head

(7613, 3)


<bound method NDFrame.head of          id                                               text  target
0         1  Our Deeds are the Reason of this #earthquake M...       1
1         4             Forest fire near La Ronge Sask. Canada       1
2         5  All residents asked to 'shelter in place' are ...       1
3         6  13,000 people receive #wildfires evacuation or...       1
4         7  Just got sent this photo from Ruby #Alaska as ...       1
...     ...                                                ...     ...
7608  10869  Two giant cranes holding a bridge collapse int...       1
7609  10870  @aria_ahrary @TheTawniest The out of control w...       1
7610  10871  M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...       1
7611  10872  Police investigating after an e-bike collided ...       1
7612  10873  The Latest: More Homes Razed by Northern Calif...       1

[7613 rows x 3 columns]>

In [11]:
test.head

<bound method NDFrame.head of          id                                               text
0         0                 Just happened a terrible car crash
1         2  Heard about #earthquake is different cities, s...
2         3  there is a forest fire at spot pond, geese are...
3         9           Apocalypse lighting. #Spokane #wildfires
4        11      Typhoon Soudelor kills 28 in China and Taiwan
...     ...                                                ...
3258  10861  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259  10865  Storm in RI worse than last hurricane. My city...
3260  10868  Green Line derailment in Chicago http://t.co/U...
3261  10874  MEG issues Hazardous Weather Outlook (HWO) htt...
3262  10875  #CityofCalgary has activated its Municipal Eme...

[3263 rows x 2 columns]>

In [12]:
# # Perform one-hot encoding
# one_hot = pd.get_dummies(df['keyword'], prefix='keyword')

# # Concatenate the one-hot encoded DataFrame with the original DataFrame
# df = pd.concat([df, one_hot], axis=1)

# # Drop the original 'countries' column
# df.drop(['keyword'], axis=1, inplace=True)

In [13]:
df.head()


,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [14]:
wordnet_lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

In [15]:
def modification_tweet(tweet):
    s=re.sub('[^a-zA-Z]',' ',tweet)
    s=s.lower()
    s=s.split()
    s=[ps.stem(word) for word in s if not word in set(stopwords.words('english'))]
    s=' '.join(s)
    return s

In [16]:
print(set(stopwords.words('english')))

{'a', "couldn't", 'it', 'we', 've', 'further', 'because', 'yourselves', 'between', 'up', 'this', 'do', 'just', 'when', "you'll", 'does', 'me', 'were', 'down', 'he', 'hasn', 'has', 'doing', "don't", 'or', 'shouldn', 'itself', 'so', 'more', "should've", 'once', 'there', 'herself', 'about', 'ours', 'didn', "didn't", 'isn', 'm', "it's", 'hers', 'by', 'are', 'that', "needn't", 'wasn', 'if', "isn't", 'your', 'out', 'their', 'here', 'ain', 'been', 'did', 'both', 'mustn', 'very', 're', 'was', 'i', 'its', 'couldn', 'but', 'hadn', 'ma', 'don', 'some', 'such', 'my', 'from', 'd', 'with', 'as', "you've", 'any', 'her', 'while', 'than', 'them', 'who', 'what', 'nor', 'an', "haven't", 'then', 'each', "hasn't", 'have', 'of', 'all', 'having', 'weren', 'myself', 'she', "wouldn't", 'no', 'during', 's', 'be', "mustn't", 'you', 'own', 'himself', 'shan', 'these', 'for', 'which', 'can', 'll', 'the', 'yourself', 'whom', 'why', 'theirs', 'our', 'mightn', 'had', 'ourselves', 'is', 'off', 'doesn', 'under', "you're

In [17]:
df["text"]=df['text'].apply(modification_tweet)
test["text"]=test['text'].apply(modification_tweet)

In [18]:
# corpus=[]
# for string in df["text"]:
#     s=string.split()
#     corpus.append(s)

# model=Word2Vec(corpus,min_count=1)
# # Access the vocabulary using index_to_key and key_to_index
# index_to_key = model.wv.index_to_key  # List of words
# key_to_index = model.wv.key_to_index  # Dictionary mapping words to indices

In [19]:
# def Word2Vector(modified_tweet):
#     s=modified_tweet.split()
#     # vec_size=wv.vector_size
#     res=np.zeros(100)
#     n=0
#     for w in s:
#         if w in index_to_key:
#             n+=1
#             word_index = key_to_index[w]
#             res=res+model.wv.vectors[word_index]
#     res=res/n
#     return res
    

In [20]:
# df['vector']=df['text'].apply(Word2Vector)

In [21]:
# y=df['target']
# df.drop(['target'], axis=1, inplace=True)

In [22]:
# x = df['vector'].to_list()

In [23]:
df.head()

,id,text,target
0,1,deed reason earthquak may allah forgiv us,1
1,4,forest fire near la rong sask canada,1
2,5,resid ask shelter place notifi offic evacu she...,1
3,6,peopl receiv wildfir evacu order california,1
4,7,got sent photo rubi alaska smoke wildfir pour ...,1


In [24]:
# df.drop(['text'], axis=1, inplace=True)
# df.drop(['vector'], axis=1, inplace=True)

In [25]:
# df_x = pd.DataFrame(x)

# # Optionally, you can specify column names for the DataFrame
# # If each vector has the same number of dimensions, you can name the columns like 'dim_0', 'dim_1', ...
# # For example, if you have 100-dimensional vectors:
# column_names = [f'dim_{i}' for i in range(100)]
# df_x.columns = column_names

In [26]:
# df=df.join(df_x)

In [27]:
df.head()

,id,text,target
0,1,deed reason earthquak may allah forgiv us,1
1,4,forest fire near la rong sask canada,1
2,5,resid ask shelter place notifi offic evacu she...,1
3,6,peopl receiv wildfir evacu order california,1
4,7,got sent photo rubi alaska smoke wildfir pour ...,1


In [28]:
# df['dim_90']

In [29]:
# df=df*1
# df.drop(['id'], axis=1, inplace=True)

In [30]:


# Split the training data into training and validation sets (80% train, 20% validation)
X_train, X_valid, y_train, y_valid = train_test_split(
    df['text'],  # Input features (preprocessed text)
    df['target'],  # Target variable
    test_size=0.2,  # Validation set size
    random_state=42  # Seed for reproducibility
)


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000,ngram_range=(1, 2))  # Maximum number of features,  
                                   # Include unigrams and bigrams

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the validation data
X_valid_tfidf = tfidf_vectorizer.transform(X_valid)

# Transform the test data (using the same vectorizer)
X_test_tfidf = tfidf_vectorizer.transform(test['text'])


In [32]:
X_train_tfidf.shape

(6090, 5000)

In [33]:
df.shape

(7613, 3)

In [34]:


# # Initialize the Logistic Regression model
# model = LogisticRegression()

# # Train the model on the TF-IDF features and target variable
# model.fit(X_train_tfidf,y_train)

# # Predict on the validation set
# y_pred = model.predict(X_valid_tfidf)
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


# # Make predictions on the test data
# test_predictions = model.predict(X_test_tfidf)


# # Calculate accuracy
# accuracy = accuracy_score(y_valid, y_pred)
# accuracy

In [35]:
svm_classifier = SVC()
# logistic_regression_classifier = LogisticRegression()
# # multinomial_nb_classifier = MultinomialNB()

svm_classifier.fit(X_train_tfidf, y_train)
# logistic_regression_classifier.fit(X_train, y_train)
# # multinomial_nb_classifier.fit(X_train, y_train)

SVC()

In [36]:
# conf_matrix = confusion_matrix(y_valid, y_pred)
# # sb.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=axes[0, 1])
# axes[0, 1].set_title('Confusion Matrix')
# axes[0, 1].set_xlabel('Predicted Labels')
# axes[0, 1].set_ylabel('True Labels')

In [37]:
svm_predictions = svm_classifier.predict(X_valid_tfidf)

# logistic_regression_predictions = logistic_regression_classifier.predict(X_valid_tfidf)
# multinomial_nb_predictions = multinomial_nb_classifier.predict(X_test)
# Make predictions on the test data
test_predictions_svm = svm_classifier.predict(X_test_tfidf)


In [38]:


# Calculate confusion matrices
svm_cm = confusion_matrix(y_valid, svm_predictions)
# lr_cm = confusion_matrix(y_valid, y_pred)
# multinomial_nb_cm = confusion_matrix(y_test, multinomial_nb_predictions)


print("Confusion Matrix for SVM:")
print(svm_cm)
# print("nConfusion Matrix for Logistic Regression:")
# print(lr_cm)
# # print("\nConfusion Matrix for Multinomial Naive Bayes:")
# print(multinomial_nb_cm)


Confusion Matrix for SVM:
[[772 102]
 [204 445]]


In [39]:
# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(y_test,logistic_regression_predictions)

In [40]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_valid,svm_predictions)
accuracy

0.799080761654629

In [41]:

# Create a DataFrame for the submission
# submission_df = pd.DataFrame({'id': test['id'], 'target': test_predictions})
# submission_df.to_csv('sample_submission.csv', index=False)

In [43]:



# Create a DataFrame for the submission
submission_df_csv = pd.DataFrame({'id': test['id'], 'target': test_predictions_svm})
submission_df_csv.to_csv('sample_submissionsvm.csv', index=False)